<a href="https://colab.research.google.com/github/submarinejuice/CP322-Final-Project-Group-9/blob/Michelle-Main/cp322_FINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Uploading Data CSV from repo so we always have it and dont have to manually import
Michelle's addition

In [ ]:

import os

REPO_URL = "https://github.com/submarinejuice/CP322-Final-Project-Group-9"
REPO_NAME = "CP322-Final-Project-Group-9"

if not os.path.exists(REPO_NAME):
    # First time in this Colab session: clone the repo
    !git clone {REPO_URL}
else:
    # Repo already there in this runtime: pull latest changes
    %cd {REPO_NAME}
    !git pull
    %cd /content

# Move into repo so relative paths work
%cd /content/{REPO_NAME}


Cloning into 'CP322-Final-Project-Group-9'...
remote: Enumerating objects: 35, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 35 (delta 8), reused 12 (delta 3), pack-reused 0 (from 0)
Receiving objects: 100% (35/35), 345.71 KiB | 4.38 MiB/s, done.
Resolving deltas: 100% (8/8), done.
/content/CP322-Final-Project-Group-9


2ndary Dataset I used:

In [ ]:
import pandas as pd
import re

print("Current directory:", os.getcwd())
print("Repo contents:", os.listdir())
print("DATASET contents:", os.listdir("DATASET"))

df = pd.read_csv("DATASET/AE_investment_dataset.csv")
df.head()
df.info()
df.isna().mean().sort_values().head(20)
df.columns.tolist()
for c in df.columns:
    print(c)





Current directory: /content/CP322-Final-Project-Group-9
Repo contents: ['cp322_FINAL.ipynb', '.git', 'README.md', 'DATASET']
DATASET contents: ['AE_investment_dataset.csv']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Columns: 364 entries, Participant_code to SCR_AnticipatoryS4_T10
dtypes: float64(356), int64(5), object(3)
memory usage: 85.4+ KB
Participant_code
Age
Gender
Nationality
Ethnicity
Played_stock_market
Played_in_years
Played_how_often
Stock_amount_S1_T1
Stock_amount_S1_T2
Stock_amount_S1_T3
Stock_amount_S1_T4
Stock_amount_S1_T5
Stock_amount_S1_T6
Stock_amount_S1_T7
Stock_amount_S1_T8
Stock_amount_S1_T9
Stock_amount_S1_T10
Stock_amount_S2_T1
Stock_amount_S2_T2
Stock_amount_S2_T3
Stock_amount_S2_T4
Stock_amount_S2_T5
Stock_amount_S2_T6
Stock_amount_S2_T7
Stock_amount_S2_T8
Stock_amount_S2_T9
Stock_amount_S2_T10
Stock_amount_S3_T1
Stock_amount_S3_T2
Stock_amount_S3_T3
Stock_amount_S3_T4
Stock_amount_S3_T5
Stock_amount_S3_T6
Stock_amount_S3_T7
Stock_amo

##Quick note bc I didn't know what PANAS meant:
- PANAS refers to the Positive and Negative Affect Schedule, a widely used psychological scale that measures an individual's mood by assessing both positive and negative emotions. Developed in 1988 by Watson, Clark, and Tellegen, it's a 20-item self-report measure used in research and clinical settings to gauge how frequently someone experiences emotions like interest, joy, enthusiasm (positive affect) versus feelings of distress, sadness, and nervousness (negative affect).
## How it works
- 20 items: The scale consists of 20 words that describe different feelings and emotions.
- Two dimensions: These items are separated into two subscales: one for positive affect (PA) and one for negative affect (NA).
- Rating scale: Participants rate how they felt about each item over a specific time frame (e.g., "right now," "today," "over the past few weeks") on a 5-point scale.
- Scoring: Each positive and negative item is scored individually. The total positive score and total negative score are then calculated. A higher positive score indicates more positive affect, while a higher negative score indicates more negative affect.

Building a per-trial table with:
1. inputs per step:
  - money_in_stocks
  - mean_return
  - stock_fluctuation
  - scr_anticipatory
2. Static inputs:
3. Target
  - Whether they invested in the stock (money_in_stocks > 0 -> 1 else 0)

#Kaggle Dataset for WESAD

-- note: ref doc later

In [ ]:
# Install dependencies as needed:
# pip install kagglehub[hf-datasets]
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Set the path to the file you'd like to load
file_path = ""

# Load the latest version
hf_dataset = kagglehub.load_dataset(
  KaggleDatasetAdapter.HUGGING_FACE,
  "orvile/wesad-wearable-stress-affect-detection-dataset",
  file_path,
  # Provide any additional arguments like
  # sql_query, hf_kwargs, or pandas_kwargs. See
  # the documenation for more information:
  # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterhugging_face
)

print("Hugging Face Dataset:", hf_dataset)

In [ ]:
import re
import pandas as pd

# 1. ID & static columns we carry along
id_cols = ["Participant_code", "Age", "Gender", "Nationality", "Ethnicity", "Played_stock_market"]

# 2. Find all trial-level columns by prefix
stock_cols = [c for c in df.columns if c.startswith("Money_in_stocks_S")]
scr_cols   = [c for c in df.columns if c.startswith("SCR_AnticipatoryS")]
ret_cols   = [c for c in df.columns if c.startswith("Mean_Return_S")]
fluc_cols  = [c for c in df.columns if c.startswith("stock_fluctuation_S")]

print("n_stock_cols:", len(stock_cols))
print("n_scr_cols:", len(scr_cols))
print("n_return_cols:", len(ret_cols))
print("n_fluctuation_cols:", len(fluc_cols))


n_stock_cols: 40
n_scr_cols: 40
n_return_cols: 36
n_fluctuation_cols: 36


In [ ]:
import re
import pandas as pd

# df is already loaded:
# df = pd.read_csv("DATASET/AE_investment_dataset.csv")

# 0. ID & static columns
id_cols = ["Participant_code", "Age", "Gender", "Nationality", "Ethnicity", "Played_stock_market"]

# 1. Grab trial-level columns by prefix
stock_cols = [c for c in df.columns if c.startswith("Money_in_stocks_S")]
scr_cols   = [c for c in df.columns if c.startswith("SCR_AnticipatoryS")]
ret_cols   = [c for c in df.columns if c.startswith("Mean_Return_S")]
fluc_cols  = [c for c in df.columns if c.startswith("stock_fluctuation_S")]

print("n_stock_cols:", len(stock_cols))
print("n_scr_cols:", len(scr_cols))
print("n_return_cols:", len(ret_cols))
print("n_fluctuation_cols:", len(fluc_cols))

# 2. Map (session, trial) -> column name
def build_lookup(cols, prefix):
    lookup = {}
    for c in cols:
        # e.g. Money_in_stocks_S1_T3  →  session=1, trial=3
        m = re.match(rf"{re.escape(prefix)}(\d+)_T(\d+)$", c)
        if m:
            s = int(m.group(1))   # session number
            t = int(m.group(2))   # trial number within session
            lookup[(s, t)] = c
    return lookup

stock_map = build_lookup(stock_cols, "Money_in_stocks_S")
scr_map   = build_lookup(scr_cols,   "SCR_AnticipatoryS")
ret_map   = build_lookup(ret_cols,   "Mean_Return_S")
fluc_map  = build_lookup(fluc_cols,  "stock_fluctuation_S")

print("number of keys in stock_map:", len(stock_map))
print("some keys from stock_map:", list(stock_map.items())[:5])


n_stock_cols: 40
n_scr_cols: 40
n_return_cols: 36
n_fluctuation_cols: 36
number of keys in stock_map: 40
some keys from stock_map: [((1, 1), 'Money_in_stocks_S1_T1'), ((1, 2), 'Money_in_stocks_S1_T2'), ((1, 3), 'Money_in_stocks_S1_T3'), ((1, 4), 'Money_in_stocks_S1_T4'), ((1, 5), 'Money_in_stocks_S1_T5')]


In [ ]:
rows = []

for _, row in df.iterrows():
    # carry participant-level info
    base = {col: row[col] for col in id_cols}

    for (s, t) in sorted(stock_map.keys()):
        rec = dict(base)
        rec["session"] = s
        rec["trial_in_session"] = t
        rec["global_trial"] = (s - 1) * 10 + t  # 1..40

        rec["money_in_stocks"] = row[stock_map[(s, t)]]
        rec["scr_anticipatory"] = row[scr_map[(s, t)]]

        # Some (session, trial) combos might not have return/fluctuation
        rec["mean_return"] = row[ret_map[(s, t)]] if (s, t) in ret_map else pd.NA
        rec["stock_fluctuation"] = row[fluc_map[(s, t)]] if (s, t) in fluc_map else pd.NA

        rows.append(rec)

long_df = pd.DataFrame(rows)

# Target: did they invest at all?
long_df["invested"] = (long_df["money_in_stocks"] > 0).astype(int)

print(long_df.shape)
long_df.head()
print(long_df["invested"].value_counts())


(1200, 14)
invested
1    1079
0     121
Name: count, dtype: int64


In [ ]:


import pandas as pd
import numpy as np

# Drop rows where our key features are missing
key_features = ["scr_anticipatory", "mean_return", "stock_fluctuation", "money_in_stocks"]

clean_df = long_df.dropna(subset=key_features).copy()

# Convert types to numeric
for col in key_features:
    clean_df[col] = pd.to_numeric(clean_df[col], errors='coerce')

# Drop again if any become NA
clean_df = clean_df.dropna(subset=key_features)

# Target
y = clean_df["invested"].astype(int)

# Features: minimal baseline
X = clean_df[["scr_anticipatory", "mean_return", "stock_fluctuation"]]

print("Clean shape:", clean_df.shape)
X.head()


Clean shape: (1080, 14)


,scr_anticipatory,mean_return,stock_fluctuation
1,0.028,0.045,2.0
2,0.232,-0.046,1.0
3,0.954,0.370,2.0
4,0.000,0.046,2.0
5,0.858,0.001,1.0


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, classification_report

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

# Baseline model
clf = LogisticRegression()
clf.fit(X_train_scaled, y_train)

y_pred = clf.predict(X_test_scaled)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("F1-score:", f1_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 0.8888888888888888
F1-score: 0.9411764705882353
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        24
           1       0.89      1.00      0.94       192

    accuracy                           0.89       216
   macro avg       0.44      0.50      0.47       216
weighted avg       0.79      0.89      0.84       216



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
from sklearn.neural_network import MLPClassifier

# Scale features again (safe to reuse)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

# MLP model: small but strong
mlp = MLPClassifier(
    hidden_layer_sizes=(32, 16),
    activation='relu',
    solver='adam',
    max_iter=500,
    random_state=42
)

mlp.fit(X_train_scaled, y_train)

y_pred_mlp = mlp.predict(X_test_scaled)

print("MLP Accuracy:", accuracy_score(y_test, y_pred_mlp))
print("MLP F1:", f1_score(y_test, y_pred_mlp))
print(classification_report(y_test, y_pred_mlp))


MLP Accuracy: 0.9629629629629629
MLP F1: 0.9791666666666666
              precision    recall  f1-score   support

           0       0.83      0.83      0.83        24
           1       0.98      0.98      0.98       192

    accuracy                           0.96       216
   macro avg       0.91      0.91      0.91       216
weighted avg       0.96      0.96      0.96       216



/usr/local/lib/python3.12/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


MLP Cell


In [ ]:
from sklearn.neural_network import MLPClassifier

#scale data again
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

# MLP model: small but strong
mlp = MLPClassifier(
    hidden_layer_sizes=(32, 16),
    activation='relu',
    solver='adam',
    max_iter=500,
    random_state=42
)

mlp.fit(X_train_scaled, y_train)

y_pred_mlp = mlp.predict(X_test_scaled)

print("MLP Accuracy:", accuracy_score(y_test, y_pred_mlp))
print("MLP F1:", f1_score(y_test, y_pred_mlp))
print(classification_report(y_test, y_pred_mlp))


MLP Accuracy: 0.9629629629629629
MLP F1: 0.9791666666666666
              precision    recall  f1-score   support

           0       0.83      0.83      0.83        24
           1       0.98      0.98      0.98       192

    accuracy                           0.96       216
   macro avg       0.91      0.91      0.91       216
weighted avg       0.96      0.96      0.96       216



/usr/local/lib/python3.12/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


In [ ]:
import numpy as np

# Work on a copy, sorted by participant + time
scr_df = clean_df.sort_values(["Participant_code", "session", "trial_in_session"]).copy()

grp = scr_df.groupby("Participant_code")

# Participant-level mean/std and z-score
scr_df["scr_mean_p"] = grp["scr_anticipatory"].transform("mean")
scr_df["scr_std_p"]  = grp["scr_anticipatory"].transform("std")
scr_df["scr_z"] = (scr_df["scr_anticipatory"] - scr_df["scr_mean_p"]) / scr_df["scr_std_p"]

# Lags within each participant
scr_df["scr_lag1"] = grp["scr_anticipatory"].shift(1)
scr_df["scr_lag2"] = grp["scr_anticipatory"].shift(2)

# Changes vs previous trials
scr_df["scr_delta1"] = scr_df["scr_anticipatory"] - scr_df["scr_lag1"]
scr_df["scr_delta2"] = scr_df["scr_anticipatory"] - scr_df["scr_lag2"]

# Short-term rolling window stats (window=3 trials)
scr_df["scr_roll_mean3"] = grp["scr_anticipatory"].transform(
    lambda x: x.rolling(window=3, min_periods=1).mean()
)
scr_df["scr_roll_std3"] = grp["scr_anticipatory"].transform(
    lambda x: x.rolling(window=3, min_periods=1).std()
)

# Replace NaNs from lags / std=0 with 0 for now
scr_feature_cols = [
    "scr_anticipatory",
    "scr_z",
    "scr_lag1", "scr_lag2",
    "scr_delta1", "scr_delta2",
    "scr_roll_mean3", "scr_roll_std3",
]

scr_df[scr_feature_cols] = scr_df[scr_feature_cols].fillna(0.0)

print("SCR feature shape:", scr_df[scr_feature_cols].shape)
scr_df[scr_feature_cols + ["invested"]].head()


SCR feature shape: (1080, 8)


,scr_anticipatory,scr_z,scr_lag1,scr_lag2,scr_delta1,scr_delta2,scr_roll_mean3,scr_roll_std3,invested
1,0.028,-0.515138,0.000,0.000,0.000,0.000,0.028000,0.000000,1
2,0.232,0.294617,0.028,0.000,0.204,0.000,0.130000,0.144250,1
3,0.954,3.160515,0.232,0.028,0.722,0.926,0.404667,0.486548,1
4,0.000,-0.626281,0.954,0.232,-0.954,-0.232,0.395333,0.497531,1
5,0.858,2.779454,0.000,0.954,0.858,-0.096,0.604000,0.525277,1


In [ ]:
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score

# ==== Prepare data ====
X_scr = scr_df[scr_feature_cols].values.astype("float32")
y_scr = scr_df["invested"].values.astype("int64")

X_train, X_val, y_train, y_val = train_test_split(
    X_scr, y_scr, test_size=0.2, random_state=42, stratify=y_scr
)

# Standardize SCR features
scaler_scr = StandardScaler()
X_train_scaled = scaler_scr.fit_transform(X_train)
X_val_scaled   = scaler_scr.transform(X_val)

X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_val_tensor   = torch.tensor(X_val_scaled, dtype=torch.float32)
y_val_tensor   = torch.tensor(y_val, dtype=torch.long)

train_ds = TensorDataset(X_train_tensor, y_train_tensor)
val_ds   = TensorDataset(X_val_tensor, y_val_tensor)

train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)
val_loader   = DataLoader(val_ds, batch_size=64, shuffle=False)

input_dim = X_train_tensor.shape[1]
emb_dim   = 16

# ==== Physiology encoder ====
class PhysioEncoder(nn.Module):
    def __init__(self, in_dim, hidden_dim=32, emb_dim=16):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, emb_dim),
            nn.ReLU()
        )

    def forward(self, x):
        return self.net(x)

# Full model: encoder + classifier head
class PhysioModel(nn.Module):
    def __init__(self, encoder, emb_dim):
        super().__init__()
        self.encoder = encoder
        self.classifier = nn.Linear(emb_dim, 2)

    def forward(self, x):
        z = self.encoder(x)        # [batch, emb_dim]
        logits = self.classifier(z)  # [batch, 2]
        return logits

encoder = PhysioEncoder(input_dim, hidden_dim=32, emb_dim=emb_dim)
model = PhysioModel(encoder, emb_dim)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# ==== Training loop ====
n_epochs = 30

for epoch in range(1, n_epochs + 1):
    model.train()
    train_losses = []

    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)

        optimizer.zero_grad()
        logits = model(xb)
        loss = criterion(logits, yb)
        loss.backward()
        optimizer.step()

        train_losses.append(loss.item())

    # Validation
    model.eval()
    all_preds = []
    all_true  = []
    with torch.no_grad():
        for xb, yb in val_loader:
            xb, yb = xb.to(device), yb.to(device)
            logits = model(xb)
            preds = torch.argmax(logits, dim=1)
            all_preds.append(preds.cpu())
            all_true.append(yb.cpu())

    all_preds = torch.cat(all_preds).numpy()
    all_true  = torch.cat(all_true).numpy()

    acc = accuracy_score(all_true, all_preds)
    f1  = f1_score(all_true, all_preds)

    if epoch % 5 == 0 or epoch == 1:
        print(f"Epoch {epoch:02d} | "
              f"train_loss={np.mean(train_losses):.4f} | "
              f"val_acc={acc:.3f} | val_f1={f1:.3f}")


Epoch 01 | train_loss=0.5734 | val_acc=0.889 | val_f1=0.941
Epoch 05 | train_loss=0.3696 | val_acc=0.889 | val_f1=0.941
Epoch 10 | train_loss=0.3537 | val_acc=0.889 | val_f1=0.941
Epoch 15 | train_loss=0.3572 | val_acc=0.889 | val_f1=0.941
Epoch 20 | train_loss=0.3466 | val_acc=0.889 | val_f1=0.941
Epoch 25 | train_loss=0.3586 | val_acc=0.889 | val_f1=0.941
Epoch 30 | train_loss=0.3478 | val_acc=0.889 | val_f1=0.941
